<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 03 - Creating and Querying a Data Mart

In this tutorial you will learn how to create and query a Data Mart using Virtualized Data in a SQL Server big data cluster. 

Wide World Importers is interested in ingesting the data from web logs from an HDFS source where they have been streamed. They want to be able to analyze the traffic to see if there is a pattern in time, products or locations. 

The web logs, however, are refreshed periodically. WWI would like to keep the logs in local storage to do deeper analysis. 

In this Jupyter Notebook you'll create a location to store the log files as a SQL Server Table in the SQL Data Pool, and then fill it by creating an External Table that reads HDFS.

In [1]:
USE [WideWorldImporters-Full];
GO

  IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlDataPool')
      CREATE EXTERNAL DATA SOURCE SqlDataPool
      WITH (LOCATION = 'sqldatapool://controller-svc:8080/datapools/default');

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

In [2]:
CREATE EXTERNAL TABLE [web_clickstream_clicks_data_pool]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
    DATA_SOURCE = SqlDataPool,
    DISTRIBUTION = ROUND_ROBIN
    );
GO

Commands completed successfully.

Total execution time: 00:00:04.490

In [12]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_clickstreams_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [web_clickstreams_hdfs]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/web_logs',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:00.001

In [13]:
BEGIN
   INSERT INTO web_clickstream_clicks_data_pool
   SELECT wcs_click_date_sk
    , wcs_click_time_sk 
    , wcs_sales_sk 
    , wcs_item_sk 
    , wcs_web_page_sk  
    , wcs_user_sk 
     FROM web_clickstreams_hdfs
END

(6770549 rows affected)

Total execution time: 00:00:34.004

In [15]:
SELECT count(*) AS TotalRecords FROM [dbo].[web_clickstream_clicks_data_pool]
SELECT TOP 10 * FROM [dbo].[web_clickstream_clicks_data_pool]

(1 row affected)

(10 rows affected)

Total execution time: 00:00:03.269

TotalRecords
6770549


wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
37980,15568,NULL,14420,16,NULL
37980,75463,NULL,3728,16,NULL
37980,3663,NULL,16099,16,NULL
37980,45935,NULL,6411,16,NULL
37980,50658,NULL,11448,16,NULL
37980,52813,NULL,7567,16,NULL
37980,2736,NULL,16244,16,NULL
37980,18942,NULL,4177,16,NULL
37980,83424,NULL,11922,16,NULL
37980,83566,NULL,10624,16,NULL


## Next Steps: Continue on to Working with Spark and ETL

Now you're ready to open the next Python Notebook - `notebook_04.ipynb` - to learn how to create and work with Spark and Extracting, Transforming and Loading data.